<a href="https://colab.research.google.com/github/samshugart/samshugart.github.io/blob/master/CNACT_para_extract_Alpha_3_27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyMuPDF
!pip install nltk
import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 79.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
pip install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Import libraries 
import fitz
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize

# Function to extract text from a PDF using fitz
def extract_text_from_pdf_fitz(pdf_path):
    with fitz.open(pdf_path) as pdf_file:
        text = ""
        for page in pdf_file:
            text += page.get_text('text')
    return text

# Function to preprocess the text
def preprocess_text(text):
    words = word_tokenize(text)
    words = [word.lower() for word in words]
    return ' '.join(words)

# Function to find words around a keyword
def find_words_around_keyword(keyword, text, num_words=60):
    pattern = r'((?:\w+\W+){%d})\b%s\b((?:\W+\w+){%d})' % (num_words, keyword, num_words)
    matches = re.findall(pattern, text, re.IGNORECASE)
    return matches

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# List of keywords to search for
#keywords = ['net zero', 'carbon neutral', 'task force on climate financial disclosure', 'TCFD', 'governance', 'steering', 'disclosures', 'compensation', 'transition plan', '1.5OC', '2OC', 'risk management process', 'physical risk', 'transition risk', 'target', 'targets', 'scope 1', 'scope 2', 'scope 3', 'carbon price', 'offsetting', 'insetting', 'double counting', 'carbon removal', 'sbti', 'science-based targeting initiative']
keywords = ['net zero', 'carbon neutral', 'task force on climate financial disclosure', 'TCFD']


# Load the PDF text and preprocess it
pdf_path = 'Blackrock_TCFD_2021.pdf'
text_fitz = extract_text_from_pdf_fitz(pdf_path)
preprocessed_text = preprocess_text(text_fitz)

FileNotFoundError: ignored

In [ ]:
top_terms_dict = {}
my_stop_words = list(TfidfVectorizer(stop_words='english').get_stop_words()) + ['climate'] +['change']
vectorizer = TfidfVectorizer(stop_words=my_stop_words, max_features=5000)
for keyword in keywords:
    # Find the words around the keyword
    matches = find_words_around_keyword(keyword, preprocessed_text)
    df = pd.DataFrame(matches, columns=['before', 'after'])
    df['result'] = df['before']  + df['after']
    keyword_string = ' '.join(df["result"].astype(str).tolist())

    # Add the keyword to the custom stop words list
    my_stop_words_with_keyword = my_stop_words + [keyword.replace(' ', '_')]

    # Check if the vocabulary is empty after adding custom stop words
    vectorizer_with_stopwords = TfidfVectorizer(stop_words=my_stop_words_with_keyword, max_features=5000)
    vectorizer_with_stopwords.fit_transform([' '])  # fit on an empty string
    if not vectorizer_with_stopwords.vocabulary_:  # if the vocabulary is empty
        keyword_string += ' dummyterm'  # add a dummy term to the keyword string

    # Fit and transform the text data
    tfidf = vectorizer.fit_transform([keyword_string])

    # Get the feature names and their corresponding TF-IDF scores
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores = tfidf.toarray()

    # Get the top 10 most important terms and their corresponding TF-IDF scores
    sorted_scores = sorted(list(zip(feature_names, tfidf_scores[0])), key=lambda x: -x[1])
    top_terms = [f"{term} :{round(score, 2)}\n" for term, score in sorted_scores[:10]]

    # Add the top terms to the dictionary
    top_terms_dict[keyword] = ' '.join(top_terms)

Keyword string for 'net zero': encourage further standardization of global sustainability disclosures . as advocates for transparency , we recognize the importance of blackrock meeting the same standards of disclosure that we ask of the companies our clients are invested in . tcfd is a key mechanism through which blackrock can provide transparency to stakeholders on how we are managing climate risk and addressing the global transition to  net zero  within our business . but it ’ s also an important mechanism through which we can contribute to the evolving global dialogue on what informative and decision-useful climate-related disclosure looks like to investors . this year , blackrock announced a series of commitments in support of clients as they prepare their portfolios for a net zero economy . getting to net zero requires risk management platform , aladdin® , which combines comprehensive portfolio management , trading , and risk reporting tools with sophisticated risk analytics on a 

ValueError: ignored

ValueError: ignored

In [ ]:
# Create a DataFrame from the dictionary
df = pd.DataFrame.from_dict(top_terms_dict, orient='index', columns=['top terms'])

# Add a column for the keywords
df['keywords'] = df.index

# Reorder the columns
df = df[['keywords', 'top terms']]

# Save the DataFrame to a CSV file
df.to_csv('keyword_top_terms.csv', index=False)